In [12]:
#Code to help get all csv files into a useable format for making into figures
import pandas as pd
import numpy as np
import sys
import os
thisDir = os.path.expanduser('~/Desktop/MSC_Alexis/analysis/')
#directory with random permutation of features
ranDir= thisDir + 'output/randomIdx/'
#directory for all regular analysis
outDir = thisDir + 'output/mL/results/ridge/acc/'
taskList=['mixed','mem','motor']
subList=['MSC01','MSC02','MSC03','MSC04','MSC05','MSC06','MSC07','MSC10']
#all permutations of features
rand_CV=pd.DataFrame()
rand_SS=pd.DataFrame()
rand_DS=pd.DataFrame()
rand_BS=pd.DataFrame()
subDir=thisDir+'output/subNetwork/'
randDir=thisDir+'output/randomIdx/results/ridge/acc/'
netList=['unassign','default', 'visual','fp','dan','van','salience','co','sm','sm-lat', 'auditory','pmn','pon']
feature=[14808,10824,8736, 4620,5264,3151, 484, 4060, 2375,316, 564,45,21]
netRoi=dict([('unassign',14808),('default', 10824),('visual',8736),('fp', 4620),('dan',5264),('van',3151),('salience', 484),('co', 4060),('sm', 2375),('sm-lat', 316),('auditory', 564),('pmn',45),('pon',21)])

features=[10,20,30,40,50,60,70,80,90,100,200,300,400,500,1000,1500,2000,2500,3000,4000,5000,10000,15000,20000,30000,40000,50000]
for feat in features:
    #CV
    CVfile=pd.read_csv(ranDir+'/results/ridge/acc/CV/'+str(feat)+'.csv')
    CVfile.drop(columns='sub', inplace=True)
    CVfile=CVfile.melt(value_vars=['mixed', 'motor','mem'], var_name='Task', value_name='Accuracy')
    CVfile['Features']=feat
    rand_CV=pd.concat([rand_CV,CVfile])
    #SS
    SSfile=pd.read_csv(ranDir+'/results/ridge/acc/SS/'+str(feat)+'.csv')
    SSfile['Features']=feat
    rand_SS=pd.concat([rand_SS,SSfile])
    #DS
    DSfile=pd.read_csv(ranDir+'/results/ridge/acc/DS/'+str(feat)+'.csv')
    DSfile['Features']=feat
    rand_DS=pd.concat([rand_DS,DSfile])
    #BS
    BSfile=pd.read_csv(ranDir+'/results/ridge/acc/BS/'+str(feat)+'.csv')
    BSfile['Features']=feat
    rand_BS=pd.concat([rand_BS,BSfile])
    
rand_CV.to_csv(ranDir+'/results/ridge/acc/CV/all.csv')
rand_SS.to_csv(ranDir+'/results/ridge/acc/SS/all.csv')
rand_DS.to_csv(ranDir+'/results/ridge/acc/DS/all.csv')
rand_BS.to_csv(ranDir+'/results/ridge/acc/BS/all.csv')


#allFolds
#DS
motor=pd.DataFrame()
mixed=pd.DataFrame()
mem=pd.DataFrame()
for train in subList:
    for test in subList:
        if train==test:
            continue
        #motor
        tmpmotor=pd.read_csv(outDir+'DS/folds/'+train+test+'motor.csv',usecols=[1,2,3])
        motor=pd.concat([motor,tmpmotor])
        #mixed
        tmpmix=pd.read_csv(outDir+'DS/folds/'+train+test+'mixed.csv',usecols=[1,2,3])
        mixed=pd.concat([mixed,tmpmix])
        #mem
        tmpmem=pd.read_csv(outDir+'DS/folds/'+train+test+'mem.csv',usecols=[1,2,3])
        mem=pd.concat([mem,tmpmem])
        
motMix=pd.merge(motor,mixed,on=['train', 'test'])        
DS_folds=pd.merge(motMix,mem,on=['train', 'test'])   
DS_folds.rename(columns={'mixed':'Mixed','motor':'Motor','mem':'Memory'}, inplace=True)
DS_violin=pd.melt(DS_folds, id_vars=['train','test'],value_vars=['Motor','Mixed','Memory'], var_name='task', value_name='acc')
DS_violin.to_csv(outDir+'DS/folds/allFolds.csv', index=False)

#SScondensed 
for sub in subList:
    sub_tmp=pd.DataFrame()
    for train in taskList:
        for test in taskList:
            if train==test:
                continue
            tmp=pd.read_csv(outDir+'SS/folds/'+train+test+sub+'.csv',usecols=[1,2,3])
            sub_tmp=pd.concat([sub_tmp,tmp])
    #return sub_tmp
    sub_tmp.to_csv(outDir+'SS/folds/'+sub+'.csv', index=False)

#BScondensed 
for train_sub in subList:
    subF=pd.DataFrame()
    for test_sub in subList:
        if train_sub==test_sub:
            continue
        for train_task in taskList:
            for test_task in taskList:
                if train_task==test_task:
                    continue
                tmp=pd.read_csv(outDir+'BS/folds/'+train_sub+test_sub+train_task+test_task+'.csv', usecols=[1,2,3,4,5])
                subF=pd.concat([subF, tmp])
    subF.to_csv(outDir+'BS/folds/'+train_sub+'.csv', index=False)
#SS once all folds per tasks are condensed into subs
SS=pd.DataFrame()
for sub in subList:
    tmp=pd.read_csv(outDir+'SS/folds/'+sub+'.csv')
    tmp=pd.melt(tmp, id_vars=['train','test'], value_vars=[sub], var_name='sub', value_name='acc')
    SS=pd.concat([SS,tmp])
SS.to_csv(outDir+'SS/folds/allFolds.csv', index=False)
#BS once all folds are condensed into subs
BS=pd.DataFrame()
subList=['MSC01','MSC02','MSC03','MSC04','MSC05','MSC06','MSC07','MSC10']
for sub in subList:
    tmp=pd.read_csv(outDir+'BS/folds/'+sub+'.csv')
    BS=pd.concat([BS,tmp])
BS.to_csv(outDir+'BS/folds/allFolds.csv', index=False)
#reformedFolds CV in results.py code



#Random sampling and feature selection calculate z scores
#subnetworks overlapped with random network sampling
SS_nets=pd.DataFrame()
DS_nets=pd.DataFrame()
CV_nets=pd.DataFrame()
BS_nets=pd.DataFrame()
for i in netList:
    tmpSS=pd.read_csv(subDir+i+'/results/ridge/acc/SS/acc.csv')
    #tmpSS['Analysis']='ridge'
    tmpSS['Network']=i
    tmpSS['Features']=netRoi[i]
    SS_nets=pd.concat([SS_nets,tmpSS])
    
    tmpDS=pd.read_csv(subDir+i+'/results/ridge/acc/DS/acc.csv')
    tmpDS['Network']=i
    tmpDS['Features']=netRoi[i]
    DS_nets=pd.concat([DS_nets,tmpDS])
    
    tmpBS=pd.read_csv(subDir+i+'/results/ridge/acc/BS/acc.csv')
    tmpBS['Network']=i
    tmpBS['Features']=netRoi[i]
    BS_nets=pd.concat([BS_nets,tmpBS])
    
    tmpCV=pd.read_csv(subDir+i+'/results/ridge/acc/CV/acc.csv')
    tmpCV=tmpCV.melt(id_vars='sub', value_vars=['mixed', 'motor','mem'], var_name='Task', value_name='Accuracy')
    tmpCV['Network']=i
    tmpCV['Features']=netRoi[i]
    CV_nets=pd.concat([CV_nets,tmpCV])
    
    
SS_nets.drop(columns='Unnamed: 0', inplace=True)
DS_nets.drop(columns='Unnamed: 0', inplace=True)
BS_nets.drop(columns='Unnamed: 0', inplace=True)
CV_nets.rename(columns={'Accuracy':'acc'},inplace=True)


#netRoi
SStotal=[]
DStotal=[]
CVtotal=[]
BStotal=[]
for i in netList:
    #CV
    CVtmp=CV_nets[(CV_nets['Network']==i)].acc.values
    CVacc=CVtmp.mean()
    CVtotal.append(CVacc)
    #SS
    SStmp=SS_nets[(SS_nets['Network']==i)].acc.values
    SSacc=SStmp.mean()
    SStotal.append(SSacc)
    #DS
    DStmp=DS_nets[(DS_nets['Network']==i)].acc.values
    DSacc=DStmp.mean()
    DStotal.append(DSacc)
    #BS
    BStmp=BS_nets[(BS_nets['Network']==i)].acc.values
    BSacc=BStmp.mean()
    BStotal.append(BSacc)

CV_mu=pd.DataFrame({'Network':netList, 'Acc':CVtotal,'Features':feature})
SS_mu=pd.DataFrame({'Network':netList, 'Acc':SStotal,'Features':feature})
DS_mu=pd.DataFrame({'Network':netList, 'Acc':DStotal,'Features':feature})
BS_mu=pd.DataFrame({'Network':netList, 'Acc':BStotal,'Features':feature})

#random samples
rand_CV=pd.read_csv(randDir+'CV/all.csv', usecols=[1,2,3])
rand_SS=pd.read_csv(randDir+'SS/all.csv',usecols=[1,2,3,4,5])
rand_DS=pd.read_csv(randDir+'DS/all.csv',usecols=[1,2,3,4,5])
rand_BS=pd.read_csv(randDir+'BS/all.csv',usecols=[1,2,3,4,5,6])


#zscore from the null model
rand_CV_mu=rand_CV['Accuracy'].mean()
rand_CV_sd=rand_CV['Accuracy'].std()

rand_SS_mu=rand_SS['acc'].mean()
rand_SS_sd=rand_SS['acc'].std()

rand_DS_mu=rand_DS['acc'].mean()
rand_DS_sd=rand_DS['acc'].std()

rand_BS_mu=rand_BS['acc'].mean()
rand_BS_sd=rand_BS['acc'].std()

CV_mu['z_score']=(CV_mu['Acc']-rand_CV_mu)/(rand_CV_sd)
SS_mu['z_score']=(SS_mu['Acc']-rand_SS_mu)/(rand_SS_sd)
DS_mu['z_score']=(DS_mu['Acc']-rand_DS_mu)/(rand_DS_sd)
BS_mu['z_score']=(BS_mu['Acc']-rand_BS_mu)/(rand_BS_sd)


CV_mu.to_csv(randDir+'CV/zscore_all.csv',index=False)
SS_mu.to_csv(randDir+'SS/zscore_all.csv',index=False)
DS_mu.to_csv(randDir+'DS/zscore_all.csv',index=False)
BS_mu.to_csv(randDir+'BS/zscore_all.csv',index=False)